# Cumulative event counter

### This notebook counts the cumulative spatio-temporal distribution of cellular events in the local neighbourhood of a series of apoptosis events

1. Load dictionary of ~apoptoses~ events and their corresponding times to base the spatiotemporal scans around 
2. Load corresponding tracking data and specify scan parameters
3. Iteratively calculate spatiotemporal distribution of cellular events around apoptoses, saving each distribution out a list of events/cell apperances

In [1]:
import calculate_radial_analysis as calculate
import dataio

import glob
import os
import re
from tqdm.auto import tqdm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from natsort import natsorted
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
import importlib

## 1) Event dictionary loader

Specify cell type to filter out unwanted apoptoses, want end result of event_dict that takes the format of :`event_dict = {event_ID:event_time}` 

where `event_ID` takes the format of: `GV{experiment_ID}_Pos{position_ID}_{cell_ID}_{cell_type}` 

For example: 

```
event_dict = {'GV0794_Pos4_393_GFP': '766',
             'GV0794_Pos7_364_GFP': '550',
             'GV0794_Pos7_457_GFP': '805',
             'GV0794_Pos7_369_GFP': '833',
             'GV0794_Pos7_292_GFP': '884',
             'GV0795_Pos4_358_GFP': '628',
             'GV0795_Pos4_18_GFP': '855',
             'GV0795_Pos4_1356_GFP': '992',
             'GV0795_Pos4_359_GFP': '1133',
             'GV0796_Pos4_542_GFP': '581',
             'GV0796_Pos4_1334_GFP': '719',
             'GV0796_Pos4_1586_GFP': '785'}
 ```
 
Function to apoptosis event dictionary from preexisting data store:

`dataio.apoptosis_list_loader(path_to_apop_lists, cell_type)`

Where the `path_to_apop_lists` is the directory containing the apoptosis information and the `cell_type` is either `'Scr'` or `'WT'`.

In [3]:
path_to_apop_lists = '/home/nathan/data/kraken/ras/experiment_information/extrusions/'#'/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/final/all'
event_dict = dataio.apoptosis_list_loader(path_to_apop_lists, 'Scr')
event_dict

{'{"ND0025_Pos7_930_RFP":': '"1030",'}

or if event dictionary already compiled

In [4]:
event_dict = dataio.event_dict_loader('/home/nathan/data/kraken/ras/experiment_information/extrusions/extrusion_dict.json')
event_dict

{'ND0025_Pos7_930_RFP': '1030',
 'ND0025_Pos7_958_RFP': '1136',
 'ND0025_Pos5_1_RFP': '420',
 'ND0014_Pos5_11_RFP': '194',
 'ND0014_Pos5_488_RFP_frag': '276',
 'ND0014_Pos5_1317_RFP_frag': '592',
 'ND0014_Pos5_2535_RFP_frag': '898',
 'ND0014_Pos7_1_RFP_frag': '145',
 'ND0014_Pos8_482_RFP': '900',
 'ND0014_Pos8_481_RFP': '1426',
 'ND0014_Pos9_1_RFP': '877',
 'ND0017_Pos7_1012_RFP_frag': '679',
 'ND0017_Pos7_547_RFP': '615',
 'ND0017_Pos7_814_RFP': '775',
 'ND0017_Pos7_1627_RFP_frag': '775',
 'ND0017_Pos7_1627_RFP': '952',
 'ND0021_Pos4_4_RFP': '860',
 'ND0021_Pos7_321_RFP': '1151',
 'ND0021_Pos7_320_RFP': '1139',
 'ND0021_Pos7_177_RFP': '1181',
 'ND0021_Pos7_178_RFP': '1146',
 'ND0021_Pos7_507_RFP': '1234',
 'ND0021_Pos7_506_RFP': '1234',
 'ND0021_Pos7_598_RFP': '1293',
 'ND0021_Pos7_599_RFP': '1293',
 'ND0022_Pos3_4_RFP_frag': '31',
 'ND0022_Pos3_907_RFP_frag': '821',
 'ND0022_Pos3_910_RFP_frag': '821',
 'ND0022_Pos3_940_RFP_frag': '821',
 'ND0022_Pos3_840_RFP_frag': '890',
 'ND0022_Po

In [7]:
### creating a control measure by swapping RFP IDs with GFP
import random
control_dict = {}
for i in event_dict:
    control_dict[i.replace('RFP', 'GFP')] = int(random.uniform(0,1000))

In [8]:
control_dict

{'ND0025_Pos7_930_GFP': 114,
 'ND0025_Pos7_958_GFP': 926,
 'ND0025_Pos5_1_GFP': 820,
 'ND0014_Pos5_11_GFP': 117,
 'ND0014_Pos5_488_GFP_frag': 757,
 'ND0014_Pos5_1317_GFP_frag': 525,
 'ND0014_Pos5_2535_GFP_frag': 974,
 'ND0014_Pos7_1_GFP_frag': 811,
 'ND0014_Pos8_482_GFP': 431,
 'ND0014_Pos8_481_GFP': 583,
 'ND0014_Pos9_1_GFP': 411,
 'ND0017_Pos7_1012_GFP_frag': 621,
 'ND0017_Pos7_547_GFP': 576,
 'ND0017_Pos7_814_GFP': 931,
 'ND0017_Pos7_1627_GFP_frag': 337,
 'ND0017_Pos7_1627_GFP': 565,
 'ND0021_Pos4_4_GFP': 450,
 'ND0021_Pos7_321_GFP': 840,
 'ND0021_Pos7_320_GFP': 554,
 'ND0021_Pos7_177_GFP': 449,
 'ND0021_Pos7_178_GFP': 793,
 'ND0021_Pos7_507_GFP': 940,
 'ND0021_Pos7_506_GFP': 841,
 'ND0021_Pos7_598_GFP': 139,
 'ND0021_Pos7_599_GFP': 38,
 'ND0022_Pos3_4_GFP_frag': 520,
 'ND0022_Pos3_907_GFP_frag': 78,
 'ND0022_Pos3_910_GFP_frag': 734,
 'ND0022_Pos3_940_GFP_frag': 389,
 'ND0022_Pos3_840_GFP_frag': 581,
 'ND0022_Pos3_1158_GFP_frag': 945,
 'ND0022_Pos3_1197_GFP_frag': 246,
 'ND0022_Pos3

## 2) Load corresponding cell tracking data and specifiy other parameters

`dataio.hdf5_file_finder(parent_folder_hdf5)` loads a list of all hdf5 files when supplied with the parent directory of all the experiments. This relies on a typical sub-parent directory structure of `'GV****/Pos*/*aligned/HDF/segmented.hdf5'` (aligned folder optional)

In [10]:
### load tracking data
parent_folder_hdf5 = '/home/nathan/data/kraken/ras/'
hdf5_file_list = dataio.hdf5_file_finder(parent_folder_hdf5)

### specify output path and neighbourhood scan parameters
radius, t_range = 1600, 1600
output_path = f'./ras_radial_analysis_beta/control'
Path(output_path).mkdir(parents=True, exist_ok=True)

In [8]:
importlib.reload(calculate)

<module 'calculate_radial_analysis' from '/home/nathan/analysis/cell-comp-analysis/radial_analysis/calculate_radial_analysis.py'>

In [10]:
calculate.radial_scan('WT', ## 'Scr',
    'DIVIDE', 
    event_dict, 
    hdf5_file_list, 
    radius, 
    t_range, 
   # num_bins, 
    output_path)

1600 1600


  0%|          | 0/91 [00:00<?, ?it/s]

[INFO][2022/05/27 07:29:49 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5...
[INFO][2022/05/27 07:29:49 PM] Loading tracks/obj_type_1


ID ND0025_Pos7_930_RFP experiment: ND0025/Pos7/
Loading ND0025/Pos7/
Btrack version no.: 0.5.0


[INFO][2022/05/27 07:29:50 PM] Loading objects/obj_type_1 (197115, 5) (197115 filtered: None)
[INFO][2022/05/27 07:29:53 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5
[INFO][2022/05/27 07:29:53 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5...
[INFO][2022/05/27 07:29:53 PM] Loading tracks/obj_type_2
[INFO][2022/05/27 07:29:53 PM] Loading objects/obj_type_2 (12464, 5) (12464 filtered: None)
[INFO][2022/05/27 07:29:53 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5
[INFO][2022/05/27 07:29:53 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos5/tracks.h5...
[INFO][2022/05/27 07:29:53 PM] Loading tracks/obj_type_1


...

[INFO][2022/05/28 10:50:37 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5...
[INFO][2022/05/28 10:50:37 AM] Loading tracks/obj_type_1


ID ND0025_Pos6_696_RFP_frag experiment: ND0025/Pos6/
Loading ND0025/Pos6/
Btrack version no.: 0.5.0


[INFO][2022/05/28 10:50:37 AM] Loading objects/obj_type_1 (392143, 5) (392143 filtered: None)
[INFO][2022/05/28 10:50:43 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5
[INFO][2022/05/28 10:50:43 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5...
[INFO][2022/05/28 10:50:43 AM] Loading tracks/obj_type_2
[INFO][2022/05/28 10:50:43 AM] Loading objects/obj_type_2 (5950, 5) (5950 filtered: None)
[INFO][2022/05/28 10:50:43 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded ND0025/Pos6/
ID ND0025_Pos6_634_RFP experiment: ND0025/Pos6/


(91,
 [],
 ['ND0025_Pos7_930_RFP raw lists saved out successfully',
  'ND0025_Pos7_958_RFP raw lists saved out successfully',
  'ND0025_Pos5_1_RFP raw lists saved out successfully',
  'ND0014_Pos5_11_RFP raw lists saved out successfully',
  'ND0014_Pos5_488_RFP_frag raw lists saved out successfully',
  'ND0014_Pos5_1317_RFP_frag raw lists saved out successfully',
  'ND0014_Pos5_2535_RFP_frag raw lists saved out successfully',
  'ND0014_Pos7_1_RFP_frag raw lists saved out successfully',
  'ND0014_Pos8_482_RFP raw lists saved out successfully',
  'ND0014_Pos8_481_RFP raw lists saved out successfully',
  'ND0014_Pos9_1_RFP raw lists saved out successfully',
  'ND0017_Pos7_1012_RFP_frag raw lists saved out successfully',
  'ND0017_Pos7_547_RFP raw lists saved out successfully',
  'ND0017_Pos7_814_RFP raw lists saved out successfully',
  'ND0017_Pos7_1627_RFP_frag raw lists saved out successfully',
  'ND0017_Pos7_1627_RFP raw lists saved out successfully',
  'ND0021_Pos4_4_RFP raw lists sav

In [11]:
### creating a control measure by swapping RFP IDs with GFP
for j in [0,1,2,3]:
    control_dict = {}
    for i in event_dict:
        control_dict[i.replace('RFP', 'GFP')] = int(random.uniform(0,1000))
    output_path = f'./ras_radial_analysis_beta/control{j}'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    calculate.radial_scan('WT', ## 'Scr',
        'DIVIDE', 
        control_dict, 
        hdf5_file_list, 
        radius, 
        t_range, 
       # num_bins, 
        output_path)

1600 1600


  0%|          | 0/91 [00:00<?, ?it/s]

[INFO][2022/06/01 02:55:37 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5...
[INFO][2022/06/01 02:55:37 PM] Loading tracks/obj_type_1


ID ND0025_Pos7_930_GFP experiment: ND0025/Pos7/
Loading ND0025/Pos7/
Btrack version no.: 0.5.0


[INFO][2022/06/01 02:55:37 PM] Loading objects/obj_type_1 (197115, 5) (197115 filtered: None)
[INFO][2022/06/01 02:55:41 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5
[INFO][2022/06/01 02:55:41 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5...
[INFO][2022/06/01 02:55:41 PM] Loading tracks/obj_type_2
[INFO][2022/06/01 02:55:41 PM] Loading objects/obj_type_2 (12464, 5) (12464 filtered: None)
[INFO][2022/06/01 02:55:41 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5
[INFO][2022/06/01 02:55:41 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos5/tracks.h5...
[INFO][2022/06/01 02:55:41 PM] Loading tracks/obj_type_1


...

Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded ND0025/Pos3/
Focal time not in frame
ID ND0025_Pos4_1275_GFP_frag experiment: ND0025/Pos4/
Loading ND0025/Pos4/
Btrack version no.: 0.5.0


[INFO][2022/06/02 02:07:02 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos4/tracks.h5
[INFO][2022/06/02 02:07:02 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos4/tracks.h5...
[INFO][2022/06/02 02:07:02 AM] Loading tracks/obj_type_2
[INFO][2022/06/02 02:07:02 AM] Loading objects/obj_type_2 (10302, 5) (10302 filtered: None)
[INFO][2022/06/02 02:07:02 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos4/tracks.h5
[INFO][2022/06/02 02:07:03 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5...
[INFO][2022/06/02 02:07:03 AM] Loading tracks/obj_type_1
[INFO][2022/06/02 02:07:03 AM] Loading objects/obj_type_1 (392143, 5) (392143 filtered: None)


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded ND0025/Pos4/
Focal time not in frame
ID ND0025_Pos4_26_GFP experiment: ND0025/Pos4/
Focal time not in frame
ID ND0025_Pos4_207_GFP experiment: ND0025/Pos4/
Focal time not in frame
ID ND0025_Pos6_696_GFP_frag experiment: ND0025/Pos6/
Loading ND0025/Pos6/
Btrack version no.: 0.5.0


[INFO][2022/06/02 02:07:11 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5
[INFO][2022/06/02 02:07:11 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5...
[INFO][2022/06/02 02:07:11 AM] Loading tracks/obj_type_2
[INFO][2022/06/02 02:07:11 AM] Loading objects/obj_type_2 (5950, 5) (5950 filtered: None)
[INFO][2022/06/02 02:07:11 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos6/tracks.h5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded ND0025/Pos6/
Focal time not in frame
ID ND0025_Pos6_634_GFP experiment: ND0025/Pos6/
Focal time not in frame


In [ ]:
### creating a control measure by swapping RFP IDs with GFP
for j in [4,5,6,7,8,9,10,11,12]:
    control_dict = {}
    for i in event_dict:
        control_dict[i.replace('RFP', 'GFP')] = int(random.uniform(0,1000))
    output_path = f'./ras_radial_analysis_beta/control{j}'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    calculate.radial_scan('WT', ## 'Scr',
        'DIVIDE', 
        control_dict, 
        hdf5_file_list, 
        radius, 
        t_range, 
       # num_bins, 
        output_path)

1600 1600


  0%|          | 0/91 [00:00<?, ?it/s]

[INFO][2022/06/02 02:07:12 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5...
[INFO][2022/06/02 02:07:12 AM] Loading tracks/obj_type_1
[INFO][2022/06/02 02:07:12 AM] Loading objects/obj_type_1 (197115, 5) (197115 filtered: None)


ID ND0025_Pos7_930_GFP experiment: ND0025/Pos7/
Loading ND0025/Pos7/
Btrack version no.: 0.5.0


[INFO][2022/06/02 02:07:15 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5
[INFO][2022/06/02 02:07:15 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5...
[INFO][2022/06/02 02:07:15 AM] Loading tracks/obj_type_2
[INFO][2022/06/02 02:07:15 AM] Loading objects/obj_type_2 (12464, 5) (12464 filtered: None)
[INFO][2022/06/02 02:07:15 AM] Closing HDF file: /home/nathan/data/kraken/ras/ND0025/Pos7/tracks.h5
[INFO][2022/06/02 02:07:15 AM] Opening HDF file: /home/nathan/data/kraken/ras/ND0025/Pos5/tracks.h5...
[INFO][2022/06/02 02:07:15 AM] Loading tracks/obj_type_1
[INFO][2022/06/02 02:07:16 AM] Loading objects/obj_type_1 (419463, 5) (419463 filtered: None)


...

Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded ND0024/Pos3/


[INFO][2022/06/02 04:21:53 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0024/Pos4/tracks.h5...
[INFO][2022/06/02 04:21:53 PM] Loading tracks/obj_type_1
[INFO][2022/06/02 04:21:53 PM] Loading objects/obj_type_1 (586398, 5) (586398 filtered: None)


ID ND0024_Pos4_276_GFP experiment: ND0024/Pos4/
Loading ND0024/Pos4/
Btrack version no.: 0.5.0


[INFO][2022/06/02 04:22:03 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0024/Pos4/tracks.h5
[INFO][2022/06/02 04:22:03 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0024/Pos4/tracks.h5...
[INFO][2022/06/02 04:22:03 PM] Loading tracks/obj_type_2
[INFO][2022/06/02 04:22:03 PM] Loading objects/obj_type_2 (33390, 5) (33390 filtered: None)
[INFO][2022/06/02 04:22:03 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0024/Pos4/tracks.h5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded ND0024/Pos4/
Focal time not in frame
ID ND0024_Pos4_1349_GFP experiment: ND0024/Pos4/
Focal time not in frame
ID ND0024_Pos4_1570_GFP experiment: ND0024/Pos4/
Focal time not in frame
ID ND0024_Pos4_1881_GFP experiment: ND0024/Pos4/
Focal time not in frame
ID ND0024_Pos4_1306_GFP experiment: ND0024/Pos4/


#### switching the coords of the output files and switching to feathers

In [40]:
fns = glob.glob('./ras_radial_analysis_beta/csv/*/*.csv')

In [50]:
for file in tqdm(fns):
    apop_ID = file.split('/')[-1].split('_N_')[0]
    apo_t, apo_x, apo_y = [int(re.search(r'txy_(\d+)_(\d+)_(\d+)', file)[i]) for i in range(1,4)]
    df = pd.read_csv(file, names = ['Cell ID', 'Distance from apoptosis', 'Frame', 'y', 'x']) ### SWITCH COORDS
    ### tidy up dataframe
    if len(df) == 0:
        continue
    df['Cell ID'] = df['Cell ID'].str.replace('[()]', '')
    df['x'] = df['x'].str.replace('[()]', '')
    df = df.astype(int)
    ### normalise time
    df['Time since apoptosis'] = df['Frame'] - apo_t
    ### add apop ID to dataframe
    df['Focal ID'] = [apop_ID] * len(df)
    new_fn = file.replace('csv', 'feather')
    df.to_feather(new_fn)

  0%|          | 0/434 [00:00<?, ?it/s]

#### organising output into fragmented and unfragmented output

In [8]:
frag_list = [i for i in event_dict if 'frag' in i]

In [18]:
fns = glob.glob('./ras_radial_analysis_beta/all/*.csv')

In [20]:
from shutil import copy

In [23]:
for fn in fns:
    for i in frag_list:
        expt, pos, ID, _, _ = i.split('_')
        formatted_ID = f'{expt}_{pos}_mut_-{ID}'
        if formatted_ID in fn:
            new_fn = fn.replace('all', 'frag')
            copy(fn, new_fn)
        else:
            new_fn = fn.replace('all', 'unfrag')
            copy(fn, new_fn)

# Compile individual scans into cumulative dataframes 

As this is quite data intensive I am cropping the number of scans at 1000/1839 at the moment

In [ ]:
files = natsorted(glob.glob(f'{output_path}/{radius}.{t_range}/*.csv'))
event_files = [file for file in files if 'N_events' in file]

In [ ]:
event_files

In [ ]:
N_events_df = load_radial_df(event_files, 
                             R = 300, 
                             crop_amount = 20, ### how much to crop the spatial measurements to avoid boundary effects
                             streamline = False,
                            strict_filtering=True)

In [ ]:
R = 0
crop_amount = 0
x_range = range(R, 1200 - R)
y_range = range(R, 1600 - R)
file = cell_count_files[0]
    ### get apop id to load apop xyt file
apop_ID = file.split('/')[-1].split('_N_')[0]
if 'Scr' in apop_ID:
    apop_ID = apop_ID.replace('Scr_-','')+'_RFP'
if 'wt' in apop_ID:
    print('Need to configure this')
### load full apo xyt
apo_xyt_fn = os.path.join('/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/apoptotic_tracks',
                         f'{apop_ID}.csv')
apo_df = pd.read_csv(apo_xyt_fn)
del apo_df['Unnamed: 0']
### see if any of the frames of the focal cell leave the FOV
frames_outside_fov = []
for apo_x, apo_y, apo_t in zip(apo_df['x'], apo_df['y'], apo_df['t']):
    if int(apo_x) not in x_range or int(apo_y) not in  y_range:
        frames_outside_fov.append(apo_t)
#print('Frames outside the FOV for ', apop_ID, frames_outside_fov)
### if filtering is strict then exclude any focal cell that leves fov

### if filtering is present but not strict then just exclude time points outside fov
apo_t, apo_x, apo_y = [int(re.search('txy_(\d+)_(\d+)_(\d+)', file)[i]) for i in range(1,4)]
### load dataframe
df = pd.read_csv(file, names = ['Cell ID', 'Distance from apoptosis', 'Frame', 'x', 'y'])
### tidy up dataframe
df['Cell ID'] = df['Cell ID'].str.replace('[()]', '')
df['y'] = df['y'].str.replace('[()]', '')
df = df.astype(int)
### normalise time
df['Time since apoptosis'] = df['Frame'] - apo_t
### eliminate boundary counts spatially

df['Time since apoptosis (hours)'] = round(df['Time since apoptosis']*(4/60), 2)
df['Distance from apoptosis (μm)'] = round(df['Distance from apoptosis']/3, 2)
del df['Time since apoptosis'], df['Distance from apoptosis']

In [ ]:
N_cell_df = df

In [ ]:
N_cell_df

In [ ]:
N_event_df

In [ ]:
N_event_df = df

In [ ]:
N_events_df

# Distribution of events

In [ ]:
sns.histplot(N_events_df, x = 'x', y = 'y', cbar = True,)
plt.title('Spatial distribution of wild-type divisions')

#### Spatiotemporal distributions

In [ ]:
sns.histplot(N_events_df, x = 'x', y = 'Time since apoptosis', cbar = True)
plt.title('Spatio-temporal distribution of wild-type divisions')

In [ ]:
sns.histplot(N_events_df, x = 'y', y = 'Time since apoptosis', cbar = True)
plt.title('Spatio-temporal distribution of wild-type divisions')

In [ ]:
sns.histplot(N_events_df, x = 'x', y = 'Frame', cbar = True)
plt.title('Spatio-temporal distribution of wild-type divisions')

In [ ]:
sns.histplot(N_events_df, x = 'y', y = 'Frame', cbar = True)
plt.title('Spatio-temporal distribution of wild-type divisions')

In [ ]:
sns.histplot(N_events_df, x = 'Distance from apoptosis', y = 'Frame', cbar = True)
plt.title('Spatio-temporal distribution of wild-type divisions')

### Loading cell counts for probability plot

Has to be streamlined for full list

In [ ]:
cell_count_files = [file for file in files if 'N_cell' in file]

In [ ]:
cell_count_files

In [ ]:
N_cells_df = load_radial_df(cell_count_files, 
                            R = 300, 
                            crop_amount = 20, 
                            streamline = True, 
                           strict_filtering=True) ### N_cells is large so streamlining advised

In [ ]:
N_cells_df

# Plotting the ratio of N_events to N_cells (i.e. P_events) 

These graphs are not cropped or using filtered apoptoses

In [ ]:
bins = 40

In [ ]:
N_events, xedges, yedges, _ = plt.hist2d(N_event_df['Time since apoptosis (hours)'], N_event_df['Distance from apoptosis (μm)'],
                      bins=bins, cmap='RdPu')
plt.colorbar(label = 'Number of wild-type divisions')
plt.title('Distribution of wild-type divisions \naround a Scribble apoptosis')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
N_events, xedges, yedges, _ = plt.hist2d(N_cell_df['Time since apoptosis (hours)'], N_cell_df['Distance from apoptosis (μm)'],
                      bins=bins, cmap='RdPu')
plt.colorbar(label = 'Number of wild-type cells')
plt.title('Distribution of wild-type cells \naround a Scribble apoptosis')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
N_events, xedges, yedges, _ = plt.hist2d(N_events_df['Time since apoptosis'], N_events_df['Distance from apoptosis'],
                      bins=bins, cmap='Blues')
plt.colorbar(label = 'Number of wild-type divisions')
plt.title('Distribution of wild-type divisions \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
N_cells, xedges, yedges, _ = plt.hist2d(N_cells_df['Time since apoptosis'], N_cells_df['Distance from apoptosis'],
                      bins=bins, cmap='Blues')
plt.colorbar(label = 'Number of wild-type cells')
plt.title('Distribution of wild-type cells \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
P_events = N_events/N_cells
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Blues')
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)')
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### Investigating the brightest box 

2 mitosis isn't anything to be going and drawing any conclusions from?

In [ ]:
N_events.T[0,0]

In [ ]:
N_cells.T[0,0]

In [ ]:
N_events.T[0,0]/N_cells[0,0]

# Constraining the spatiotemporal range of the plots

In [ ]:
t_min, t_max = -20,20 ## in hours
R_max = 150 ## in micrometers

In [ ]:
bins = 10

In [ ]:
N_events_constrained_df = N_events_df.loc[(N_events_df['Time since apoptosis'] >= t_min) & (N_events_df['Time since apoptosis'] <= t_max) & (N_events_df['Distance from apoptosis'] <= R_max)]
N_cells_constrained_df = N_cells_df.loc[(N_cells_df['Time since apoptosis'] >= t_min) & (N_cells_df['Time since apoptosis'] <= t_max) & (N_cells_df['Distance from apoptosis'] <= R_max)]

In [ ]:
N_events_constrained_df

In [ ]:
N_events, xedges, yedges, _ = plt.hist2d(N_events_constrained_df['Time since apoptosis'], N_events_constrained_df['Distance from apoptosis'],
                      bins=bins, cmap='Blues')
plt.colorbar(label = 'Number of wild-type divisions')
plt.title('Distribution of wild-type divisions \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
N_cells, xedges, yedges, _ = plt.hist2d(N_cells_constrained_df['Time since apoptosis'], N_cells_constrained_df['Distance from apoptosis'],
                      bins=bins, cmap='Blues')
plt.colorbar(label = 'Number of wild-type cells')
plt.title('Distribution of wild-type cells \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
P_events = N_events/N_cells
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Blues')
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
N_events[0,0]

In [ ]:
N_cells[0,0]

In [ ]:
N_events[0,0]/N_cells[0,0]

Same same kinda graph as previous times I have generated it. So what makes -20 hours so important? And if I change the bounds of the plot/bins does it's importance fail?

# Cropping based on radial scan not leaving the FOV

For a radial scan of maximum extent 300 pixels (100um, ie 5 cell radii)



In [ ]:
cell_count_files

In [ ]:
N_events_df = load_radial_df(event_files,#[0:1000], 
                            R = 300, 
                            crop_amount = 20, 
                            streamline = True, 
                            strict_filtering = False)

In [ ]:
N_cells_df = load_radial_df(cell_count_files,#[0:1000], 
                            R = 300, 
                            crop_amount = 20, 
                            streamline = True, 
                            strict_filtering = False)

In [ ]:
N_events_df

In [ ]:
N_cells_df

#### Focal apoptotic scans that leave the FOV are excluded

In [ ]:
t_range = (-10,10)
R_max = 100
bins = (20, 5)
P_events, xedges, yedges, N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

## Testing filtering of time frames outside FOV

In [ ]:
N_events_df = load_radial_df(event_files[0:1000], 
                            R = 300, 
                            crop_amount = 20, 
                            streamline = True, 
                            strict_filtering = False)

In [ ]:
N_cells_df = load_radial_df(cell_count_files[0:1000],
                            R = 300, 
                            crop_amount = 20, 
                            streamline = True, 
                            strict_filtering = False)

In [ ]:
N_events_df

In [ ]:
N_cells_df

### Time points where the focal apoptotic scan leaves the FOV are excluded

In [ ]:
t_range = (-10,10)
R_max = 100
bins = (20, 5)
P_events, xedges, yedges, N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

# No filtering

In [ ]:
N_events_df = load_radial_df(event_files[0:1000], 
                            R = 0, 
                            crop_amount = 20, 
                            streamline = True, 
                            strict_filtering = False)

In [ ]:
N_events_df

In [ ]:
N_cells_df = load_radial_df(cell_count_files[0:1000], 
                            R = 0, 
                            crop_amount = 20, 
                            streamline = True, 
                            strict_filtering = False)

### All apoptoses are included regardless of whether they leave the FOV

In [ ]:
t_range = (-10,10)
R_max = 100
bins = (20, 5)
P_events, xedges, yedges, N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### Old plots showing the distribution of focal time point xy locations when they were filtered

In [ ]:
sns.histplot(N_events_apop_xy_df, x = 'x', y = 'y', cbar = True,)
plt.title(f'Spatial distribution of filtered Scr. apoptoses (R = {R})')

In [ ]:
N_apop_events

### Now filter the N_cell counts too

In [ ]:
sns.histplot(N_cells_apop_xy_df, x = 'x', y = 'y', cbar = True,)
plt.title(f'Spatial distribution of filtered Scr. apoptoses (R = {R})')

In [ ]:
N_apop_cells

In [ ]:
### checking the exclusion of apoptoses is consistent
N_apop_cells == N_apop_events

# Plotting filtered focal apoptoses

In [ ]:
bins = 40

In [ ]:
N_events, xedges, yedges, _ = plt.hist2d(N_events_df['Time since apoptosis'], N_events_df['Distance from apoptosis'],
                      bins=bins, cmap='Blues')
plt.colorbar(label = 'Number of wild-type divisions')
plt.title('Distribution of wild-type divisions \naround Scribble apoptoses (N = 1000)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
N_cells, xedges, yedges, _ = plt.hist2d(N_cells_df['Time since apoptosis'], N_cells_df['Distance from apoptosis'],
                      bins=bins, cmap='Blues')
plt.colorbar(label = 'Number of wild-type cells')
plt.title('Distribution of wild-type cells \naround Scribble apoptoses (N = 1000)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
P_events = N_events/N_cells
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Blues')
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

# Constraining the spatiotemporal range of the plots

### This binning is 2hours/25um 

In [ ]:
P_events = N_events/N_cells
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds',)
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

# Testing different bin sizes

#### Strict filtering

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround filtered Scribble apoptoses (N = 257)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### No filtering

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### Strict filtering

In [ ]:
t_range = (-5,5)
R_max = 100
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround filtered Scribble apoptoses (N = 257)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-5,5)
R_max = 100
bins = (10, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 100
bins = (10, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (20, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### Strict filtering

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (40, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround filtered Scribble apoptoses (N = 257)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (40, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (80, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (100, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 1000
bins = (10, 10)
P_events, xedges, yedges, N_events, N_cells = render(bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-4,10)
R_max = 100
bins = (20, 5)
P_events, xedges, yedges, N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### Divisions are more homogenously distributed post apoptoses, constraining focal scan plot to post apoptosis

In [ ]:
t_range = (-10,10)
R_max = 100
bins = (10, 5)
P_events, xedges, yedges, N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 457)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

# Smooth plot

In [ ]:
import scipy.ndimage as sp

#### gaussian filter

In [ ]:
smooth_P_events = sp.filters.gaussian_filter(N_events/N_cells, sigma = 4, order = 1)
pc = plt.pcolormesh(xedges, yedges, smooth_P_events.T, cmap = 'Blues', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### contourf

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2)
cmap = plt.get_cmap('PiYG')
im = ax0.pcolormesh(xedges, yedges, P_events.T, cmap=cmap, )#norm=norm)
fig.colorbar(im, ax=ax0)
ax0.set_title('pcolormesh with levels')


# contours are *point* based plots, so convert our bound into point
# centers
cf = ax1.contourf(P_events.T, #levels=levels,
                  cmap=cmap)
fig.colorbar(cf, ax=ax1)
ax1.set_title('contourf with levels')

# adjust spacing between subplots so `ax1` title and `ax0` tick labels
# don't overlap
fig.tight_layout()

plt.show()

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
P_events, xedges, yedges = render(bins = (15), t_range = (-60,20), R_max = 150)
fig, (ax0, ax1) = plt.subplots(nrows=2)
cmap = plt.get_cmap('Blues')
im = ax0.pcolormesh(xedges, yedges, P_events.T, cmap=cmap, )#norm=norm)
fig.colorbar(im, ax=ax0)
ax0.set_title('pcolormesh with levels')

levels = MaxNLocator(nbins=15).tick_values(P_events.min(), P_events.max())
# contours are *point* based plots, so convert our bound into point
# centers
cf = ax1.contourf(xedges[0:-1], yedges[0:-1], P_events.T, #levels=levels,
                  cmap=cmap)
fig.colorbar(cf, ax=ax1)
ax1.set_title('contourf with levels')

# adjust spacing between subplots so `ax1` title and `ax0` tick labels
# don't overlap
fig.tight_layout()

plt.show()

In [ ]:
P_events, xedges, yedges = render(bins = (10), t_range = (-20,20), R_max =100)
pc = plt.contourf(xedges[0:-1], yedges[0:-1], P_events.T, cmap = 'viridis', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
P_events, xedges, yedges = render(bins = (10), t_range = (-12,12), R_max =100)
pc = plt.contourf(xedges[0:-1], yedges[0:-1], P_events.T, cmap = 'viridis', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scribble apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

## Investigating the number of events in each plot

In [ ]:
ax = sns.heatmap(N_events, xticklabels=xedges, yticklabels=yedges, cmap = 'Blues', annot= True)
ax.invert_yaxis()

In [ ]:
ax = sns.heatmap(N_cells, xticklabels=xedges, yticklabels=yedges, cmap = 'Blues', annot= True)
ax.invert_yaxis()

In [ ]:
np.flipud(N_events.T)

# Control measures

### 1. Wild-type divisions around random wild-type points

I have some skepticism about this method as it seems too randomised, leading it to pick up some potential boundary effects of scanning in space and time, i.e. it's suspicious that the peaks in probability are repeated in near pre-apoptotic time.

### 2. Wild-type divisions around wild-type apoptoses

Processed

### 3. Wild-type divisions around Scr. apoptoses in uninduced experiments

Need to find Scr. apoptoses in uninduced experiments.

#### Shall I filter these control measures for scans within the FOV?

# Loading control measure #1

In [ ]:
files = natsorted(glob.glob(f'/home/nathan/data/results/radial_analysis_output/follow/raw_lists/1600.1600/wt_control_wt_div_xyt/1600.1600/*.csv'))

In [ ]:
files

In [ ]:
len(files)

### Limiting the number of control radial scans to include as 15,000 is far too many

Will pick 1839 as that's the same as the previous scans

In [ ]:
output_path = f'/home/nathan/data/results/radial_analysis_output/follow/raw_lists/1600.1600/wt_control_wt_div_xyt/'
radius, t_range = 1600,1600

In [ ]:
files = natsorted(glob.glob(f'{output_path}{radius}.{t_range}/*.csv'))[0:2000]
event_files = [file for file in files if 'N_events' in file]
cell_count_files = [file for file in files if 'N_cells' in file]

In [ ]:
for n, i in enumerate((event_files)):
    print(os.path.basename(event_files[n]))
    print(os.path.basename(cell_count_files[n]))

In [ ]:
len(event_files)

In [ ]:
control_N_event_df = load_radial_df(event_files)

In [ ]:
control_N_cell_df = load_radial_df(cell_count_files)

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (10, 10)
P_events_c, xedges, yedges,  N_events, N_cells = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events_c.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround random wild-type timepoints (N = 1000)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-20,10)
R_max = 200
bins = (10, 10)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events_c.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround random wild-type timepoints (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

#### checking against reloaded normal plot

In [ ]:
output_path = f'/home/nathan/data/results/radial_analysis_output/follow/raw_lists/1600.1600/scr_apop_wt_div_xyt/'
radius, t_range = 1600,1600

files = natsorted(glob.glob(f'{output_path}{radius}.{t_range}/*.csv'))[0:(1839*2)]
event_files = [file for file in files if 'N_events' in file]
cell_count_files = [file for file in files if 'N_cells' in file]

In [ ]:
N_events_df

In [ ]:
N_event_df = load_radial_df(event_files)

N_cell_df = load_radial_df(cell_count_files)

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround filtered Scr. apoptoses (N = 1000)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-10,10)
R_max = 200
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_event_df, N_cell_df, bins, t_range, R_max)
pc = plt.pcolormesh(xedges, yedges, P_events.T, cmap = 'Reds', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839)')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

## Probability above background

In [ ]:
t_range = (-5,5)
R_max = 100
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround filtered Scr. apoptoses (N = 1000) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-20,20)
R_max = 200
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround random Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-5,5)
R_max = 100
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_event_df, N_cell_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround random Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (0,5)
R_max = 100
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_event_df, N_cell_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-4,10)
R_max = 200
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_event_df, N_cell_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-2,8)
R_max = 60
bins = (10, 10)
P_events, xedges, yedges,  N_events, N_cells = render(N_event_df, N_cell_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-4,4)
R_max = 100
bins = (16, 4)
P_events, xedges, yedges,  N_events, N_cells = render(N_event_df, N_cell_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_N_event_df, control_N_cell_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

# Control measure #2 

### Wild-type divisions around wild-type apoptoses

In [ ]:
control_2_file_path = '/home/nathan/data/results/radial_analysis_output/follow/raw_lists/1600.1600/wt_apop_wt_div_xyt/1600.1600'

In [ ]:
files = natsorted(glob.glob(f'{control_2_file_path}/*.csv'))
event_files = [file for file in files if 'N_events' in file]
cell_count_files = [file for file in files if 'N_cells' in file]

In [ ]:
len(event_files)

In [ ]:
control_2_N_events_df = load_radial_df(event_files)

control_2_N_cells_df = load_radial_df(cell_count_files)

In [ ]:
t_range = (-12,12)
R_max = 200
bins = (10,10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_2_N_events_df, control_2_N_cells_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (-6,6)
R_max = 200
bins = (10,10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_2_N_events_df, control_2_N_cells_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

In [ ]:
t_range = (0,6)
R_max = 200
bins = (12,10)
P_events, xedges, yedges,  N_events, N_cells = render(N_events_df, N_cells_df, bins, t_range, R_max)
P_events_c, xedges, yedges,  N_events_c, N_cells_c = render(control_2_N_events_df, control_2_N_cells_df, bins, t_range, R_max)

pc = plt.pcolormesh(xedges, yedges, P_events.T - P_events_c.T, cmap = 'PiYG', )
plt.colorbar(label = 'Probability of wild-type division \n(No. divisions/No. cells)',)
plt.title('Distribution of wild-type division probability \naround Scr. apoptoses (N = 1839) \n above background')
plt.ylabel('Distance from apoptoses (μm)')
plt.xlabel('Time since apoptoses (hours)')

# Conclusion from two control measures

There seems to be a noticeable peak in probability of division in the immediate neighbourhood at approx. 2 and 5 and 10 hours beforehand... why? 

#### Trying density plots

In [ ]:
N_events_df[0:100]

#### Need to work out way to do ratio of two smoothed density plots

In [ ]:
sns.kdeplot(data=N_events_df, x="Time since apoptosis", y="Distance from apoptosis")
